# Obtención de datos - API REE

Importante:

La plataforma Microsoft Fabric transforma los datos en formato Json a formato tabular de manera automatica. Por ello, este notebook no 
maneja una forma de estructurar los datos. Con la url es suficiente.

En primer lugar, creamos un programa que nos genere un excel con el id de cada uno de los indicadores 
para los que Red Electrica ofrece información.

In [ ]:
import pandas as pd
import requests

#Codigo para obtener los id de los indicadores

#Para ver todas las filas de nuestro df
pd.set_option('display.max.rows', 2000)

#Token ESIOS (introducir tu propio token)
esios_token = ''  #Solicitado a REE

# Url base
address = "https://api.esios.ree.es/indicators"

# Llamada a la API y construccion de json
resp = requests.get(address, headers={'x-api-key' : '%s' %(esios_token),
                                             'Authorization' : 'Token token=%s' %(esios_token)}).json()

#Creamos dos listas vacías donde incorporar los datos del bucle for
name = []
id = []

# Bucle para rellenar las dos listas anteriores con los datos del nombre y el id de los indicadores de REE.
for i in range(0, len(resp['indicators'])):
    name.append(resp['indicators'][i]['name'])
    id.append(resp['indicators'][i]['id'])

# Creamos un df a partir de las listas anteriores
indicadores = pd.DataFrame({'nombre': name, 'id': id})

print(indicadores)

indicadores.to_excel("./lista_esios_indicadores.xlsx")

El excel se genera por defecto en el directorio donde tengas guardado el Notebook

## Programa con todos los inputs

Una vez conocemos los parametros necesarios para obtener nuestro indicador, tenemos que crear un programa que nos permita 
generar la url necesaria en cada caso.

En este sentido creamos un primer programa personalizable, capaz de obtener una url distinta en cada caso.
Convertimos también en json para observar si los datos que extraemos son los necesarios.

In [ ]:
# Este programa nos permite formar la url que necesitemos en cada caso.

import requests as rq

# Parametros para constrir la url
esios_token = ''
url_base = 'https://api.esios.ree.es/indicators'
indicador = input('introduce el id del indicador: ') + '?'
start_date = 'start_date=' + input('Fecha inicial Formato Y-M-D: ') + 'T24:00:00Z'
end_date = 'end_date=' + input('Fecha final Formato Y-M-D: ') + 'T24:00:00Z'
time_trunc = 'time_trunc=' + input('five_minutes, ten_minutes, fifteen_minutes, hour, day, month, year: ')
time_agg = 'time_agg=' + input('sum,average: ')
geo_ids = 'geo_ids[]=' + input('geo_id: ')
geo_trunc = 'geo_trunc=' + input('country, electric_system, autonomous_community, province, electric_subsystem, town and drainage_basin: ')

#Construccion de url
url_api = url_base + '/' + indicador + start_date + '&' + end_date + '&' + time_trunc + '&' + time_agg  + '&' + geo_ids + '&'+ geo_trunc
print(url_api)
json_file = rq.get(url_api, headers={'x-api-key' : '%s' %(esios_token),
                                             'Authorization' : 'Token token=%s' %(esios_token)}).json()
print(json_file)

## Indicador 1001

En nuestro caso, deseamos obtener el indicador 1001 para la península ibérica (sin Portugal).
Nota: El precio PVPc es el mismo en todo el territorio nacional

Podemos comentar y descomentar en la medida que lo necesitemos

En este caso, los parametros que debemos determinar son los siguientes:

indicador = 1001

start_date = 2023-01-01

end_date = None para actualizar la informacion periodicamente y obtener siempre los últimos datos disponibles

time_trunc =  hour (para calcular spread del pvpc)

time_agg = average

geo_ids = 8741 (para obtener datos de la península)

In [ ]:
# Indicador 1001 con geo_id = 8741 (peninsula)
# Generamos el json para verificar los datos

import requests as rq

# Parametros para constrir la url
esios_token = ''
url_base = 'https://api.esios.ree.es/indicators'
indicador = input('introduce el id del indicador: ') + '?'
start_date = 'start_date=' + input('Fecha inicial Formato Y-M-D: ') + 'T24:00:00Z'
# end_date = 'end_date=' + input('Fecha final Formato Y-M-D: ') + 'T24:00:00Z'
time_trunc = 'time_trunc=' + input('five_minutes, ten_minutes, fifteen_minutes, hour, day, month, year: ')
time_agg = 'time_agg=' + input('sum,average: ')
geo_ids = 'geo_ids[]=' + input('geo_id: ' )
# geo_trunc = 'geo_trunc=' + input('country, electric_system, autonomous_community, province, electric_subsystem, town and drainage_basin: ')

#Construccion de url
url_api = url_base + '/' + indicador + start_date  + '&' + time_trunc + '&' + time_agg + '&' + geo_ids
print(url_api)

#Obtención del json
json_file = rq.get(url_api, headers={'x-api-key' : '%s' %(esios_token),
                                             'Authorization' : 'Token token=%s' %(esios_token)}).json()
print(json_file)

Una vez hemos obtenido la url necesaria y hemos comprobado que los datos son correctos, es hora de importar la dirección
en Microsoft Fabric.